# Imports

In [1]:
import os
import numpy as np
import pandas as pd

# Primary analysis (3.47-day doubling time, 15% ascertainment rate) tree sizes

In [2]:
def read_clade_results(dir):
    clade_analyses_d = dict()
    for path in sorted(os.listdir(dir)): # pool all clade analyses together
        clade_analysis_path = dir + path
        key = int(path.split('_')[0])
        clade_analyses_d[key] = {'clade_sizes': [], 'subclade_sizes': []}
        for line in open(clade_analysis_path):
            l = line.strip().strip(']').split('[')
            clade_size = int(l[0].strip()) # make each clade size (including single leaves) an integer
            subclade_sizes = [int(x) for x in l[1].strip().replace(' ', '').split(',')] # put each subclade size (including single leaves) into a list
            clade_analyses_d[key]['clade_sizes'].append(clade_size)
            clade_analyses_d[key]['subclade_sizes'].append(subclade_sizes)
    return clade_analyses_d

In [3]:
clade_analyses_CC_dir = './0.28TF/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir = './0.28TF/clade_analyses_AB_polytomy_updated/'

clade_analyses_CC_d = read_clade_results(clade_analyses_CC_dir)

print('Less than 787 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) < 787 for x in clade_analyses_CC_d])/1100)) 
print('More than 1000 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) > 1000 for x in clade_analyses_CC_d])/1100))
print('More than 5000 taxa: %f' % (sum([sum(clade_analyses_CC_d[x]['clade_sizes']) > 5000 for x in clade_analyses_CC_d])/1100))

Less than 787 taxa: 0.014545
More than 1000 taxa: 0.983636
More than 5000 taxa: 0.967273


# Tree shapes and bayes factors

In [4]:
unconstrained_results = np.array([1.68,80.85, 10.32, 0.92])/100 # linB, linA, C/C, T/T
recCA_results = np.array([77.28, 8.18, 10.49, 3.71])/100 # linB, linA, C/C, T/T

In [5]:
def calculate_bf(asr_results, simulation_results):
    # Let t_p be a polytomy, t_1C be one clade, and t_2c be two clades. Note that t_p is a topology with a polytomy but is not t_1c. t_p + t_1c equals all topologies with a basal polytomy (Fig. 2a). 
    # trees are in the order
    # (t_p, t_1C, t_2C, (t_p,(t_p,t_1C,t_2C)), (t_1C,(t_p,t_1C,t_2C)), (t_2c,(t_p,t_1C,t_2C)))
    compatibility_matrix = np.array([np.array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_A
                                     np.array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_B
                                     np.array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0]), # S_CC
                                     np.array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0])] # S_TT
                                    )
                                    
    # A matrix of conditional probabilities
    # Each row stores the vector Pr(S_MRCA | \btau)
    pr_s_mrca_given_tree = np.array([x/sum(x) if sum(x) > 0 else x for x in compatibility_matrix.T]) # if tree not associated with any haplotype, just keep the row as all 0s
    
    # Order: S_A, S_B, S_{C/C}, S_{T/T}
    pr_s_mrca_given_data = np.array(asr_results)/sum(asr_results)
    unnormalized_pr_data_given_s_mrca = pr_s_mrca_given_data.copy()
    
    # FAVITES simulation information
    # the 3 trees are in the order (t_p, t_1C, t_2C)
    pr_3_topos = np.array(simulation_results)
    pr_trees_given_I1 = np.concatenate([pr_3_topos, np.array([0]*9)])
    pr_trees_given_I2 = np.concatenate([np.array([0]*3), np.outer(pr_3_topos, pr_3_topos).flatten()])
    
    # Equal prior probability of 1 or 2 intros
    pr_I1 = 0.5
    pr_I2 = 0.5
    
    pr_s_mrca_and_I1 = np.dot([np.dot(pr_s_mrca_given_tree.T[i], pr_trees_given_I1) for i in range(0,4)], pr_I1) # dot product of P(haplotype|tree) (column) and P(trees|I_n), scaled by P(I_n)
    pr_s_mrca_and_I2 = np.dot([np.dot(pr_s_mrca_given_tree.T[i], pr_trees_given_I2) for i in range(0,4)], pr_I2)

    posterior_odds = np.dot(unnormalized_pr_data_given_s_mrca, pr_s_mrca_and_I2) / np.dot(unnormalized_pr_data_given_s_mrca, pr_s_mrca_and_I1)
    prior_odds = pr_I2/pr_I1
    BF = posterior_odds/prior_odds

    return(BF)



In [6]:
def clade_analysis_updated(clade_analyses_CC_dir, clade_analyses_AB_dir, label, min_polytomy_size=100, _print_=False):
    # C/C analyses focus on 1-mutation clades
    # therefore, all descendant lineages are included:
    #   -1 = unmutated leaf
    #    1 = mutated leaf separate from a clade
    #   >1 = clade
    clade_analyses_CC_d = read_clade_results(clade_analyses_CC_dir)

    # A/B analyses focus on 2-mutation clades
    # only 2-mutation clades are included 
    clade_analyses_AB_d = read_clade_results(clade_analyses_AB_dir)
    
    # C/C analysis
    cc_count_30perc_twoPolytomies = 0 # how often are there only two 1-mutation clades, each constituting more than 30% of taxa AND with a polytomy at the base of each clade, and no other descendants from the root?
    for run in clade_analyses_CC_d:
        clade_sizes = clade_analyses_CC_d[run]['clade_sizes'] # get the clade sizes
        subclade_sizes = [len(x)>=min_polytomy_size for x in clade_analyses_CC_d[run]['subclade_sizes']] # check if each clade has a polytomy at the base
        if len(clade_sizes) == 2: # if there are more than two descendants, including individual leaves, skip
            if min(clade_sizes) > (sum(clade_sizes)*0.30): # make sure each clade is >30%
                if False not in subclade_sizes: # each clade must have a polytomy at the base
                    cc_count_30perc_twoPolytomies += 1
    
    # A/B analysis
    ab_count_30perc_twoPolytomies = 0 # interested in 2 mutations clade that are at least 30% of all taxa + has a basal polytomy + polytomy at 2 mutation clade
    lower_constraint = 0.3 # the 2-mutation clade must be at least 30% of all taxa
    upper_constraint = 0.7 # the 2-mutation clade must be at most 70% of all taxa

    for run in clade_analyses_AB_d:
        num_leaves = sum(clade_analyses_CC_d[run]['clade_sizes'])
        base_polytomy_size = len(clade_analyses_CC_d[run]['clade_sizes']) # check how many lineages descend from the root
        clade_sizes = clade_analyses_AB_d[run]['clade_sizes'] 
        subclade_sizes = clade_analyses_AB_d[run]['subclade_sizes']
        if not clade_sizes: # no 2 mutation clades
            continue
        for index, clade_size in enumerate(clade_sizes): # loop through all 2 mutation clades
            if lower_constraint*num_leaves <= clade_size <= upper_constraint*num_leaves: # clade match size restrictions
                if base_polytomy_size >= min_polytomy_size: # basal polytomy
                    if len(subclade_sizes[index]) >= min_polytomy_size: # polytomy at 2 mutation clade
                        ab_count_30perc_twoPolytomies += 1 # if all conditions are met, add 1 to the count
                        break # if one 2 mutation clade meets the conditions, break out of the loop and move on to the next run

    # polytomy: checking if there is a polytomy at the base of the tree; can do this using the C/C analysis
    min_polytomy_descendants = min_polytomy_size
    count_atLeastMinDescendants = 0
    for run in clade_analyses_CC_d: # loop through all runs
        clade_sizes = clade_analyses_CC_d[run]['clade_sizes'] # get the clade sizes
        if len(clade_sizes) >= min_polytomy_descendants:  # if there are at least X number of descendants, including individual leaves
            count_atLeastMinDescendants += 1
    polytomy_result = count_atLeastMinDescendants/1100
    
    # calculate bayes factors
    cc_result = cc_count_30perc_twoPolytomies/1100
    ab_result = ab_count_30perc_twoPolytomies/1100

    simulation_results = [polytomy_result-ab_result, ab_result, cc_result] # note that polytomy result is inclusive of ab_result, so subtract ab_result from polytomy_result to avoid double counting
    unconstrained_results = np.array([1.68, 80.85, 10.32, 0.92])/100 # linA, linB, C/C, T/T
    recCA_results = np.array([77.28, 8.18, 10.49, 3.71])/100 # linA, linB, C/C, T/T
    bf_unconstrained = calculate_bf(unconstrained_results, simulation_results)
    bf_recCA = calculate_bf(recCA_results, simulation_results)
    
    if _print_ == True:
        print('Proportion with a polytomy at the base: %f\n' % (count_atLeastMinDescendants/1100))
        print('C/C clade analysis: 2 clades, polytomy at base of each clade, each clade possessing at least 30%% of the taxa: %f\n' % (cc_count_30perc_twoPolytomies/1100))
        print('A/B clade analysis: Basal polytomy, polytomy at 2-mut clade, with the clade possessing between 30 and 70%% of the taxa: %f\n' % (ab_count_30perc_twoPolytomies/1100))
        print('Unconstrained Bayes factor: %f' % bf_unconstrained)
        print('recCA Bayes factor: %f' % bf_recCA)

    return [label, min_polytomy_size, "{:.1f}".format(cc_result*100), "{:.1f}".format(ab_result*100), "{:.1f}".format(polytomy_result*100), "{:.1f}".format(bf_unconstrained), "{:.1f}".format(bf_recCA)]



### Main result

In [7]:
# 3.5DT
clade_analyses_CC_dir = './0.28TF/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir = './0.28TF/clade_analyses_AB_polytomy_updated/'
results = clade_analysis_updated(clade_analyses_CC_dir, clade_analyses_AB_dir, '3.5 DT', min_polytomy_size=100, _print_=True)

Proportion with a polytomy at the base: 0.475455

C/C clade analysis: 2 clades, polytomy at base of each clade, each clade possessing at least 30% of the taxa: 0.000000

A/B clade analysis: Basal polytomy, polytomy at 2-mut clade, with the clade possessing between 30 and 70% of the taxa: 0.030909

Unconstrained Bayes factor: 4.154836
recCA Bayes factor: 4.264418


### All results

In [8]:
all_results = []

# 2.5DT
clade_analyses_CC_dir_25DT = './0.38TF/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir_25DT = './0.38TF/clade_analyses_AB_polytomy_updated/'
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_25DT, clade_analyses_AB_dir_25DT, '2.5 DT', min_polytomy_size=100, _print_=False))

# 3.5DT
clade_analyses_CC_dir_35DT = './0.28TF/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir_35DT = './0.28TF/clade_analyses_AB_polytomy_updated/'
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT, clade_analyses_AB_dir_35DT, '3.5 DT', min_polytomy_size=100, _print_=False))

# # 4.5DT
clade_analyses_CC_dir_45DT = './0.22TF/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir_45DT = './0.22TF/clade_analyses_AB_polytomy_updated/'
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_45DT, clade_analyses_AB_dir_45DT, '4.5 DT', min_polytomy_size=100, _print_=False)) 

# # 3.5 DT, low asc
clade_analyses_CC_dir_35DT_lowAsc = './0.295TF_0.05r/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir_35DT_lowAsc = './0.295TF_0.05r/clade_analyses_AB_polytomy_updated/'
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT_lowAsc, clade_analyses_AB_dir_35DT_lowAsc, '3.5 DT, low asc', min_polytomy_size=100, _print_=False))

# # 3.5 DT, high asc
clade_analyses_CC_dir_35DT_highAsc = './0.255TF_0.25r/clade_analyses_CC_polytomy_updated/'
clade_analyses_AB_dir_35DT_highAsc = './0.255TF_0.25r/clade_analyses_AB_polytomy_updated/'
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT_highAsc, clade_analyses_AB_dir_35DT_highAsc, '3.5 DT, high asc', min_polytomy_size=100,  _print_=False))

# 3.5DT sensitivity analyses
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT, clade_analyses_AB_dir_35DT, '3.5 DT', min_polytomy_size=20, _print_=False))
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT, clade_analyses_AB_dir_35DT, '3.5 DT', min_polytomy_size=50, _print_=False))
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT, clade_analyses_AB_dir_35DT, '3.5 DT', min_polytomy_size=200, _print_=False))
all_results.append(clade_analysis_updated(clade_analyses_CC_dir_35DT, clade_analyses_AB_dir_35DT, '3.5 DT', min_polytomy_size=500, _print_=False))



### Generate table

In [9]:
pooled_df = pd.DataFrame(all_results, columns = ['Analysis', 'Min. polytomy size', 'C/C', 'A/B', 'Polytomy', 'Unconstrained', 'recCA'])
pooled_df

,Analysis,Min. polytomy size,C/C,A/B,Polytomy,Unconstrained,recCA
0,2.5 DT,100,0.0,3.4,58.6,5.8,6.0
1,3.5 DT,100,0.0,3.1,47.5,4.2,4.3
2,4.5 DT,100,0.1,3.5,43.1,3.0,3.0
3,"3.5 DT, low asc",100,0.0,3.4,45.7,3.5,3.6
4,"3.5 DT, high asc",100,0.2,3.5,47.3,3.6,3.7
5,3.5 DT,20,0.1,5.1,60.7,4.1,4.2
6,3.5 DT,50,0.1,3.9,53.6,4.2,4.3
7,3.5 DT,200,0.0,2.1,40.7,4.5,4.6
8,3.5 DT,500,0.0,1.1,31.7,5.2,5.4


In [10]:
# pooled_df.to_csv('bayes_factors.csv', index=None)